In [1]:
!pip install groq 

zsh:1: /Users/pulkitnargotra/Library/Python/3.9/bin/pip: bad interpreter: /Library/Developer/CommandLineTools/usr/bin/python3: no such file or directory


In [2]:
import requests
import pandas as pd
import re
from datasets import load_dataset
import os

df = load_dataset("ChilleD/SVAMP")


/Users/pulkitnargotra/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/pulkitnargotra/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
test_df = df['test'].to_pandas()

In [4]:
test_df = test_df.head(300)

In [5]:
# test_df['final_answer'] = test_df['answer'].apply(lambda x: re.search(r'#### (\d+)', x).group(1) if re.search(r'#### (\d+)', x) else None)

In [6]:
correct_labels = []
for i in range(len(test_df['Answer'])):
    correct_labels.append(test_df['Answer'][i])

In [7]:
correct_labels[0]

'27'

In [8]:
chain_of_thought_prompts = []
for i in range(len(test_df)):
    question = test_df['question_concat'][i]
    answer = correct_labels[i]
    prompt =  question + "\n" + "Think step by step."
    chain_of_thought_prompts.append(prompt)

In [9]:
chain_of_thought_prompts[1]

'Paige raised 7 goldfish and 12 catfish in the pond but stray cats loved eating them. Now she has 15 left. How many fishes disappeared?\nThink step by step.'

In [10]:
os.environ['GROQ_API_KEY']='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'

In [11]:
from groq import Groq
client = Groq()

In [12]:
def answer(query):
    system_prompt = {"role": "system", "content": "You will be given a math word problem, your job is to think step by step, solve this problem and return one numerical answer. The template to answer the query is to have the last line of the output as Answer: <<<Numerical Answer>>>"}
    chat_history = [system_prompt]
    chat_history.append({"role": "user", "content": query})
    response = client.chat.completions.create(model="llama-3.1-8b-instant", messages=chat_history, temperature=0)
    return response.choices[0].message.content

In [13]:
def extract_answer_value(response):
    match = re.search(r'Answer:\s*[^0-9]*([\d]+)', response)
    if match:
        try:
            return int(match.group(1)) 
        except ValueError:
            return "X"
    return "X"  

In [14]:
# answerrr = answer(chain_of_thought_prompts[0])

In [15]:
# extract_answer_value(answerrr)

In [16]:
bad = 0
predictions_cot_model = []
for i in range(len(chain_of_thought_prompts)):
    try: 
        answerrr = answer(chain_of_thought_prompts[i])
        extracted = extract_answer_value(answerrr)
    except:
        extracted = "X"
    if(extracted == "X"):
        bad += 1
    predictions_cot_model.append(extracted)
    # print(answerrr)
    if(i%30 == 0):
        print(i)
        print(extracted)
        print("---------------")
print("Bad Responses:",bad)

0
27
---------------
30
3
---------------
60
28
---------------
90
28
---------------
120
4
---------------
150
1
---------------
180
3
---------------
210
3
---------------
240
2226
---------------
270
12
---------------
Bad Responses: 19


In [17]:
# predictions_zeroshot_model[:10]

In [18]:
correct_labels[:10]

['27', '4', '16', '64', '31', '720', '21', '64', '450', '143550']

In [19]:
bad

19

In [20]:
def accuracy(predictions,labels):
    correct = 0
    ok = 0
    for i in range(len(labels)):
        if(predictions[i] == "X"):
            continue
        if(labels[i] == str(predictions[i])):
            correct+=1
        ok+=1
    return correct*100/ok

In [21]:
print(accuracy(predictions_cot_model,correct_labels))

89.6797153024911
